
#### OBJETIVO: Generacion de mapas anuales a partir de los datos diarios

In [1]:
import numpy as np
import rasterio
import glob
from pathlib import Path
print("Libreria ok")

Libreria ok


In [ ]:

# -----------------------------
# Parámetros
# -----------------------------
year = 2024
modelo = "01-RF-CV-M1-170625-CH"
estacion = "CH"

ruta_diarios = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/salidas/"
    f"SalidasDiarias/{modelo}/{year}"
)

ruta_salida = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/Salidas/"
    f"SalidasAnuales/{modelo}"
)

Path(ruta_salida).mkdir(parents=True, exist_ok=True)

# -----------------------------
# Listar rasters diarios
# -----------------------------
lista_rasters = sorted(glob.glob(ruta_diarios + "/*.tif"))
n_files = len(lista_rasters)

print(f"Año {year} - archivos encontrados: {n_files}")

if n_files == 0:
    raise ValueError("⚠️ No se encontraron rasters diarios")

# -----------------------------
# Leer y apilar
# -----------------------------
stack = []

for path in lista_rasters:
    with rasterio.open(path) as src:
        data = src.read(1)
        profile = src.profile

        nodata = src.nodata
        if nodata is not None:
            data = np.where(data == nodata, np.nan, data)

        stack.append(data)

stack = np.stack(stack, axis=0)  # (n_dias, rows, cols)

# -----------------------------
# Promedio anual
# -----------------------------
promedio_anual = np.nanmean(stack, axis=0)

# -----------------------------
# Guardar raster anual
# -----------------------------
profile.update(
    dtype=rasterio.float32,
    count=1,
    compress="lzw",
    nodata=np.nan
)

nombre_salida = (
    f"{ruta_salida}/Promedio_anual_{year}-{modelo}.tif"
)

# with rasterio.open(nombre_salida, "w", **profile) as dst:
#     dst.write(promedio_anual.astype(np.float32), 1)

print("✔ Guardado:", nombre_salida)


Año 2024 - archivos encontrados: 365
✔ Guardado: D:/Josefina/Proyectos/ProyectoChile/CH/modelos/Salidas/SalidasAnuales/01-RF-CV-M1-170625-CH/Promedio_anual_2024-01-RF-CV-M1-170625-CH.tif
